In [ ]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_csv("data.csv")

In [3]:
df1 = data
pattern = r"\$\d+(,\d{3})*(\.\d+)??"
def clean_price(price_str):
  match = re.search(pattern, price_str)
  if match:
      cleaned_price = match.group().strip("$").replace(",", "")
      return float(cleaned_price)
  else:
      return None
df1["Price"] = df1["Price"].apply(clean_price)

df2 = df1
pattern = r"(\d+)\s*(?:day|days)\s*(?:on\s*site|site)?"

def clean_days(days_str):
  match = re.search(pattern, days_str, flags=re.IGNORECASE)
  if match:
    return int(match.group(1))
  else:
    return None
df2["No_of_days_onsite"] = df2["No_of_days_onsite"].apply(clean_days)

df3 = df2
def split_location(location_str):
  pattern = r"(.*?)\, (.*?)\, (.*)"
  match = re.match(pattern, location_str)
  if match:
    return match.groups()
  else:
    return [None, None, None]

df3[['Address', 'City', 'Zipcode']] = df3['Location'].apply(split_location).tolist()
df3 = df3.drop('Location', axis=1)
df4 = df3
df_f = df4[df4['Beds'] != '–']
df_f = df_f[df_f['Year_built'] != '–']
df_f = df_f[df_f['Baths'] != '–']
df_f['Baths'] = pd.to_numeric(df_f['Baths'], errors='coerce')
df_f.dropna(subset=['Baths'], inplace=True)
df_final = df_f
df_final['Beds'] = df_final['Beds'].astype("int64")
df_final['Baths'] = df_final['Baths'].astype("int64")
df_final['No_of_days_onsite'] = df_final['No_of_days_onsite'].astype("int64")
df_final['Sqft'] = df_final['Sqft'].str.replace(',', '')
df_final['Sqft'] = df_final['Sqft'].astype("int64")
df_final['Year_built'] = df_final['Year_built'].astype("int64")
df_final['Property_type'] = df_final['Property_type'].astype("str")
df_final['Address'] = df_final['Address'].astype("str")
df_final['City'] = df_final['City'].astype("str")
df_final['City'] = df_final['City'].str.strip()

def extract_numbers(text):
    zip_pattern = r'\b\d{5}\b'
    match = re.search(zip_pattern, text)
    if match:
        return match.group(0)
    else:
        return None
df_final['Zipcode'] = df_final['Zipcode'].apply(extract_numbers)
df_final['Zipcode'] = df_final['Zipcode'].str.join('')
df_final.dropna(subset=['Zipcode'], inplace=True)
df_final['City'] = df_final['City'].str.strip()

df_final = df_final[(df_f['Year_built'] != '0') & (~df_final['Year_built'].isna())]
df_final['Year_built'] = df_final['Year_built'].replace('–', pd.NA)
df_final.dropna(subset=['Year_built'], inplace=True)
print(df_final)
df_final.to_csv("final.csv")

         MLS_id      Price  Beds  Baths  Sqft  Year_built  \
0    ML81964272  2998888.0     4      3  2764        1986   
1    ML81964119   580000.0     2      2  1125        1986   
2    BE41057198  1588000.0     4      2  1797        1965   
3    BE41058557   699888.0     2      2  1040        1991   
4    ML81964175   799000.0     2      2   930        1987   
..          ...        ...   ...    ...   ...         ...   
801  BE41049475   685000.0     2      2   947        2000   
802  CC41049309  4195000.0     5      6  5992        1975   
803  CC41049163   375000.0     1      1   758        1974   
804  BE41043078  7200000.0     3      2  2424        1930   
814  BE41010860  1500000.0     3      2  2800        1938   

               Property_type  HOA  No_of_days_onsite  \
0    Single Family Residence  420                  1   
1                Condominium  703                  1   
2    Single Family Residence  703                  1   
3                Condominium  418          